In [299]:
from pulp import LpMaximize, LpMinimize, LpProblem, LpVariable, LpInteger, LpStatus, lpSum
import pandas as pd

In [300]:
df = pd.read_csv("nutrition_cost_table.csv")
df

,Ingredient,Protein,Fat,Fibre,Salt,CostPerGram
0,Chicken,0.10,0.08,0.001,0.002,0.013
1,Beef,0.20,0.10,0.005,0.005,0.008
2,Mutton,0.15,0.11,0.003,0.007,0.010
3,Rice,0.00,0.01,0.100,0.002,0.002
4,Wheat bran,0.04,0.01,0.150,0.008,0.005
5,Gel,0.00,0.00,0.000,0.000,0.001


In [301]:
costs = df.set_index('Ingredient')['CostPerGram'].to_dict()
protein = df.set_index('Ingredient')['Protein'].to_dict()
fat = df.set_index('Ingredient')['Fat'].to_dict()
fibre = df.set_index('Ingredient')['Fibre'].to_dict()
salt = df.set_index('Ingredient')['Salt'].to_dict()

In [302]:
# create the LP problem
problem = LpProblem(name="LP-Problem-Minimise-Objective", sense=LpMinimize)

In [303]:
# decision variables
ingredients = list(df.Ingredient)
names = LpVariable.dicts("Ingr", ingredients, lowBound=0, upBound=100)

In [304]:
# set objective function
problem += (lpSum([costs[i] * names[i] for i in ingredients]), "Total Cost of Ingredients per can")

In [305]:
# add constraints
problem += lpSum([names[i] for i in ingredients]) == 100, "PercentagesSum"
problem += lpSum([protein[i] * names[i] for i in ingredients]) >= 8.0, "ProteinRequirement"
problem += lpSum([fat[i] * names[i] for i in ingredients]) >= 6.0, "FatRequirement"
problem += lpSum([fibre[i] * names[i] for i in ingredients]) <= 2.0, "FibreRequirement"
problem += lpSum([salt[i] * names[i] for i in ingredients]) <= 0.4, "SaltRequirement"

In [306]:
# solve problem
status = problem.solve()
print("Status:", LpStatus[problem.status])
for var in problem.variables():
    print(var.name, "=", var.varValue)

Status: Optimal
Ingr_Beef = 60.0
Ingr_Chicken = 0.0
Ingr_Gel = 40.0
Ingr_Mutton = 0.0
Ingr_Rice = 0.0
Ingr_Wheat_bran = 0.0
